In [ ]:
import numpy as np
import tifffile
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
import os
os.environ['MPLBACKEND'] = 'Qt5Agg'  # Set backend before importing matplotlib
import matplotlib
matplotlib.use('Qt5Agg')  # Explicitly set backend
import matplotlib.pyplot as plt
#from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
#import PySimpleGUI as sg
from cellmincer.denoise import main as cm_denoise
print(dir(cm_denoise))
from cellmincer.models.spatial_unet_2d_temporal_denoiser import SpatialUnet2dTemporalDenoiser

# COMPLETE configuration with ALL required keys
config = {
    'type': 'spatial_unet_2d_temporal_denoiser',  # REQUIRED
    
    # Spatial U-Net parameters
    'n_global_features': 1,
    'spatial_unet_depth': 4,
    'spatial_unet_first_conv_channels': 32,
    'spatial_unet_padding': True,
    'spatial_unet_batch_norm': True,
    'spatial_unet_attention': False,
    'spatial_unet_feature_mode': 'repeat',  # 'repeat', 'once', or 'none'
    'spatial_unet_kernel_size': 3,
    'spatial_unet_n_conv_layers': 2,
    'spatial_unet_readout_kernel_size': 1,
    'spatial_unet_activation': 'relu',
    
    # Temporal denoiser parameters
    'temporal_denoiser_kernel_size': 3,
    'temporal_denoiser_conv_channels': 32,
    'temporal_denoiser_hidden_dense_layer_dims': [64],
    'temporal_denoiser_activation': 'relu',
    'temporal_denoiser_n_conv_layers': 2,  # This is used in get_temporal_order_from_config
}

# Now create the model
model = SpatialUnet2dTemporalDenoiser(config=config)
print("Model created successfully!")

def load_tif(file_path, fps):
    """
    Load imaging stack (TIFF).

    Parameters
    ----------
    file_path : str
        Path to the TIFF stack.
    fps : float
        Imaging frame rate (Hz).

    Returns
    -------
    stack : np.ndarray
        Image stack (frames, height, width).
    frame_times : np.ndarray
        Time vector for each frame (seconds).
    """
    stack = tifffile.imread(file_path)  # shape: (frames, h, w)
    n_frames = stack.shape[0]
    frame_times = np.arange(n_frames) / fps

    return stack, frame_times

FPS = 1000
tiff_stack, frame_times = load_tif("D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/014-SD10hz-70_NDTiffStack.tif", FPS)

## Train Model with Torch

In [168]:
import torch
import numpy as np
import tifffile
import matplotlib.pyplot as plt

# Load your TIFF stack
tiff_path = "D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/014-SD10hz-70_NDTiffStack.tif"
tiff_stack = tifffile.imread(tiff_path)
print(f"Loaded TIFF stack shape: {tiff_stack.shape}")

# Normalize and convert to float32
def normalize_data(data):
    data_min = data.min()
    data_max = data.max()
    if data_max - data_min > 0:
        return (data - data_min) / (data_max - data_min)
    return data

# Convert to float32 explicitly
tiff_stack_normalized = normalize_data(tiff_stack.astype(np.float32))
print(f"Data type: {tiff_stack_normalized.dtype}")

# Use a smaller subset for faster training
if tiff_stack_normalized.shape[0] > 50:
    train_data = tiff_stack_normalized[30:80]
else:
    train_data = tiff_stack_normalized

print(f"Training on data shape: {train_data.shape}")

# Create a simple frame denoiser
class SimpleFrameDenoiser(torch.nn.Module):
    def __init__(self):
        super(SimpleFrameDenoiser, self).__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(16, 32, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 64, 3, padding=1),
            torch.nn.ReLU(),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Conv2d(64, 32, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 16, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(16, 1, 3, padding=1),
            torch.nn.Sigmoid()  # Output in [0,1] range
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return self.decoder(x)

def train_simple_frame_denoiser(movie_data, num_epochs=30):
    """Train a simple frame-by-frame denoiser"""
    T, H, W = movie_data.shape
    
    model = SimpleFrameDenoiser()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.MSELoss()
    
    model.train()
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        
        # Train on individual frames
        for frame_idx in range(T):
            # Get single frame
            clean_frame = movie_data[frame_idx:frame_idx+1]  # Keep as [1, H, W]
            
            # Add noise
            noise = 0.1 * np.random.randn(*clean_frame.shape).astype(np.float32)
            noisy_frame = np.clip(clean_frame + noise, 0, 1)
            
            # Convert to tensor - ensure float32
            noisy_tensor = torch.from_numpy(noisy_frame).unsqueeze(0).float()  # [1, 1, H, W]
            clean_tensor = torch.from_numpy(clean_frame).unsqueeze(0).float()  # [1, 1, H, W]
            
            # Forward pass
            output = model(noisy_tensor)
            loss = criterion(output, clean_tensor)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        if epoch % 10 == 0:
            avg_loss = epoch_loss / T
            print(f"Epoch {epoch}, Average Loss: {avg_loss:.6f}")
    
    return model

def denoise_frames_simple(model, movie_data):
    """Denoise frames using simple model"""
    model.eval()
    T, H, W = movie_data.shape
    denoised_frames = []
    
    with torch.no_grad():
        for frame_idx in range(T):
            frame = movie_data[frame_idx:frame_idx+1]  # [1, H, W]
            input_tensor = torch.from_numpy(frame).unsqueeze(0).float()  # [1, 1, H, W]
            output_tensor = model(input_tensor)
            denoised_frame = output_tensor.numpy()[0, 0]  # [H, W]
            denoised_frames.append(denoised_frame)
    
    return np.array(denoised_frames)

# Alternative: Batch processing version (faster)
def train_batch_denoiser(movie_data, num_epochs=30):
    """Train using batch processing"""
    T, H, W = movie_data.shape
    
    model = SimpleFrameDenoiser()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.MSELoss()
    
    model.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        
        # Process in batches
        batch_size = min(8, T)  # Smaller batch size for memory
        num_batches = (T + batch_size - 1) // batch_size
        
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, T)
            
            # Get batch of frames
            clean_batch = movie_data[start_idx:end_idx]  # [batch_size, H, W]
            
            # Add noise to entire batch
            noise = 0.1 * np.random.randn(*clean_batch.shape).astype(np.float32)
            noisy_batch = np.clip(clean_batch + noise, 0, 1)
            
            # Convert to tensors - shape: [batch_size, 1, H, W]
            noisy_tensor = torch.from_numpy(noisy_batch).unsqueeze(1).float()
            clean_tensor = torch.from_numpy(clean_batch).unsqueeze(1).float()
            
            # Forward pass
            output = model(noisy_tensor)
            loss = criterion(output, clean_tensor)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if epoch % 10 == 0:
            avg_loss = total_loss / num_batches
            print(f"Epoch {epoch}, Average Loss: {avg_loss:.6f}")
    
    return model

def denoise_batch_simple(model, movie_data):
    """Denoise using batch processing"""
    model.eval()
    T, H, W = movie_data.shape
    
    denoised_frames = []
    batch_size = min(16, T)  # Process in batches
    
    with torch.no_grad():
        for start_idx in range(0, T, batch_size):
            end_idx = min(start_idx + batch_size, T)
            batch_data = movie_data[start_idx:end_idx]
            
            # Convert to tensor
            input_tensor = torch.from_numpy(batch_data).unsqueeze(1).float()  # [batch, 1, H, W]
            
            # Denoise
            output_tensor = model(input_tensor)
            denoised_batch = output_tensor.numpy()[:, 0]  # [batch, H, W]
            
            denoised_frames.extend(denoised_batch)
    
    return np.array(denoised_frames)

# Main execution
print("Training model...")
# Try batch processing for faster training
trained_model = train_batch_denoiser(train_data, num_epochs=30)

print("Denoising...")
denoised_data = denoise_batch_simple(trained_model, train_data)

# Save results
output_path = "D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/denoised_movie.tif"
tifffile.imwrite(output_path, denoised_data.astype(np.float32))
print(f"Denoised movie saved to: {output_path}")

# Save the trained model
model_save_path = "D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/trained_denoiser.pth"
torch.save(trained_model.state_dict(), model_save_path)
print(f"Model saved to: {model_save_path}")

# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original frame
axes[0].imshow(train_data[0], cmap='gray')
axes[0].set_title('Original Frame 0')
axes[0].axis('off')

# Denoised frame
axes[1].imshow(denoised_data[0], cmap='gray')
axes[1].set_title('Denoised Frame 0')
axes[1].axis('off')

# Difference
axes[2].imshow(train_data[0] - denoised_data[0], cmap='gray')
axes[2].set_title('Noise Removed')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("Done!")

Loaded TIFF stack shape: (1481, 89, 90)
Data type: float32
Training on data shape: (50, 89, 90)
Training model...
Epoch 0, Average Loss: 0.015163
Epoch 10, Average Loss: 0.006106
Epoch 20, Average Loss: 0.004018
Denoising...
Denoised movie saved to: D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/denoised_movie.tif
Model saved to: D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/trained_denoiser.pth
Done!


## Denoise Movie with CellMincer

In [169]:
class SimpleFrameDenoiser(torch.nn.Module):
    def __init__(self):
        super(SimpleFrameDenoiser, self).__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(16, 32, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 64, 3, padding=1),
            torch.nn.ReLU(),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Conv2d(64, 32, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 16, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(16, 1, 3, padding=1),
            torch.nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return self.decoder(x)

def denoise_large_video(model, video_path, output_path, chunk_size=100):
    """Denoise a large video in chunks to avoid memory issues"""
    
    # Load the full video
    print(f"Loading video: {video_path}")
    full_video = tifffile.imread(video_path)
    print(f"Full video shape: {full_video.shape}")
    
    # Normalize
    def normalize_data(data):
        data_min = data.min()
        data_max = data.max()
        if data_max - data_min > 0:
            return (data - data_min) / (data_max - data_min)
        return data
    
    full_video = normalize_data(full_video.astype(np.float32))
    
    model.eval()
    denoised_chunks = []
    
    total_frames = full_video.shape[0]
    print(f"Processing {total_frames} frames in chunks of {chunk_size}...")
    
    with torch.no_grad():
        for start_idx in range(0, total_frames, chunk_size):
            end_idx = min(start_idx + chunk_size, total_frames)
            chunk = full_video[start_idx:end_idx]
            
            print(f"Processing frames {start_idx} to {end_idx-1}...")
            
            # Convert to tensor and denoise
            input_tensor = torch.from_numpy(chunk).unsqueeze(1).float()  # [chunk_size, 1, H, W]
            output_tensor = model(input_tensor)
            denoised_chunk = output_tensor.numpy()[:, 0]  # [chunk_size, H, W]
            
            denoised_chunks.append(denoised_chunk)
            
            # Clear memory
            del input_tensor, output_tensor
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
    
    # Combine all chunks
    denoised_full = np.vstack(denoised_chunks)
    
    # Save result
    tifffile.imwrite(output_path, denoised_full.astype(np.float32))
    print(f"Denoised video saved to: {output_path}")
    
    return denoised_full

# Load your trained model
model = SimpleFrameDenoiser()
model.load_state_dict(torch.load(model_save_path))
print("Model loaded successfully!")

# Process your full video
input_video = "D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/014-SD10hz-70_NDTiffStack.tif"
output_video = "D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/014-SD10hz-70_denoised_full.tif"

denoised_full = denoise_large_video(model, input_video, output_video, chunk_size=100)
print("Done processing full video!")

Model loaded successfully!
Loading video: D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/014-SD10hz-70_NDTiffStack.tif
Full video shape: (1481, 89, 90)
Processing 1481 frames in chunks of 100...
Processing frames 0 to 99...
Processing frames 100 to 199...
Processing frames 200 to 299...
Processing frames 300 to 399...
Processing frames 400 to 499...
Processing frames 500 to 599...
Processing frames 600 to 699...
Processing frames 700 to 799...
Processing frames 800 to 899...
Processing frames 900 to 999...
Processing frames 1000 to 1099...
Processing frames 1100 to 1199...
Processing frames 1200 to 1299...
Processing frames 1300 to 1399...
Processing frames 1400 to 1480...
Denoised video saved to: D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/014-SD10hz-70_denoised_full.tif
Done processing full video!


## Segmentation with CellPose vs. Manual Segmentation

In [ ]:
from pathlib import Path
from suite2p import run_s2p, default_ops
from matplotlib.widgets import EllipseSelector
import tifffile

class Suite2pVoltagePipeline:
    """
    Complete Suite2p pipeline optimized for voltage imaging with manual fallback.
    """
    
    def __init__(self, data_path, tiff_file, output_dir='./suite2p_output'):
        """
        Parameters
        ----------
        data_path : str
            Directory containing the TIFF file
        tiff_file : str
            Name of the TIFF file (e.g., 'denoised_movie.tif')
        output_dir : str
            Where to save Suite2p results
        """
        self.data_path = Path(data_path)
        self.tiff_file = tiff_file
        self.output_dir = Path(output_dir)
        self.ops = None
        self.stat = None
        self.F = None
        self.Fneu = None
        self.spks = None
        self.iscell = None
        self.manual_mode = False  # Track if we used manual selection
        self.manual_traces = None
        self.manual_masks = None
        
    def configure_ops(self, frame_rate=1000, diameter=12, tau=0.0028, 
                     spatial_hp=100, threshold_scaling=1.0, max_iterations=50):
        """
        Configure Suite2p parameters optimized for voltage imaging.
        """
        # Start with default ops
        self.ops = default_ops()
        
        # ============ CRITICAL VOLTAGE IMAGING SETTINGS ============
        self.ops['fs'] = frame_rate
        self.ops['tau'] = tau
        self.ops['diameter'] = diameter
        self.ops['spatial_hp_detect'] = spatial_hp
        self.ops['spatial_hp_reg'] = 0
        self.ops['threshold_scaling'] = threshold_scaling
        self.ops['max_iterations'] = max_iterations
        self.ops['high_pass'] = 100
        self.ops['allow_overlap'] = True
        self.ops['max_overlap'] = 0.4
        self.ops['inner_neuropil_radius'] = 2
        self.ops['min_neuropil_pixels'] = 200
        self.ops['do_registration'] = False
        self.ops['two_step_registration'] = True
        self.ops['keep_movie_raw'] = False
        self.ops['smooth_sigma'] = 1.15
        self.ops['classifier_path'] = None
        self.ops['batch_size'] = 500
        self.ops['num_workers'] = 0
        self.ops['save_mat'] = False
        self.ops['save_NWB'] = False
        self.ops['combined'] = False
        
        print("Suite2p configured for voltage imaging:")
        print(f"  Frame rate: {frame_rate} Hz")
        print(f"  Tau: {tau} s")
        print(f"  Diameter: {diameter} pixels")
        print(f"  Spatial high-pass: {spatial_hp} pixels")
        
        return self.ops
    
    def run_suite2p_with_manual_fallback(self, enable_manual_fallback=True):
        """
        Run Suite2p with automatic fallback to manual selection.
        
        Parameters
        ----------
        enable_manual_fallback : bool
            If True, automatically fall back to manual selection when no cells are found
            
        Returns
        -------
        output_path : Path
            Path to output directory
        """
        if self.ops is None:
            print("No ops configured, using defaults for voltage imaging...")
            self.configure_ops()
        
        # Set up database
        db = {
            'data_path': [str(self.data_path)],
            'tiff_list': [str(self.data_path / self.tiff_file)],
            'save_path0': str(self.output_dir), 
        }
        
        print("\n" + "="*60)
        print("Running Suite2p...")
        print("="*60)
        print(f"Input: {self.data_path / self.tiff_file}")
        print(f"Output: {self.output_dir}")
        
        # Run Suite2p
        try:
            output_ops = run_s2p(ops=self.ops, db=db)
            self.load_results()
            
            # Check if any cells were found
            n_cells = np.sum(self.iscell[:, 0] > 0)
            print(f"Suite2p found {n_cells} cells")
            
            if n_cells == 0 and enable_manual_fallback:
                print("No cells found automatically. Falling back to manual selection...")
                self.run_manual_selection()
                self.manual_mode = True
            else:
                self.manual_mode = False
                
        except Exception as e:
            print(f"Suite2p failed with error: {e}")
            if enable_manual_fallback:
                print("Falling back to manual selection...")
                self.run_manual_selection()
                self.manual_mode = True
            else:
                raise e
        
        print("\n" + "="*60)
        print("Processing complete!")
        print("="*60)
        
        return self.output_dir / 'suite2p' / 'plane0'
    
    def run_manual_selection(self):
        """
        Run manual ROI selection when automatic detection fails.
        """
        print("\n" + "="*60)
        print("MANUAL ROI SELECTION")
        print("="*60)
        
        # Load the movie
        movie_path = self.data_path / self.tiff_file
        movie = tifffile.imread(movie_path)
        mean_image = np.max(movie, axis=0)
        
        fig, ax = plt.subplots(figsize=(12, 10))
        ax.imshow(mean_image, cmap='gray')
        ax.set_title('Click and drag to select ROIs. Press Enter when done.')
        
        rois = []  # List of (y, x, height, width)
        
        def onselect(eclick, erelease):
            """Callback for rectangle selection"""
            x1, y1 = int(eclick.xdata), int(eclick.ydata)
            x2, y2 = int(erelease.xdata), int(erelease.ydata)
            
            # Ensure coordinates are ordered
            xmin, xmax = min(x1, x2), max(x1, x2)
            ymin, ymax = min(y1, y2), max(y1, y2)
            
            width = xmax - xmin
            height = ymax - ymin
            
            rois.append((ymin, xmin, height, width))
            
            # Draw the rectangle
            rect = plt.Circle((xmin, ymin), width, height, 
                               fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(rect)
            plt.draw()
            
            print(f"Added ROI {len(rois)}: pos=({xmin}, {ymin}), size=({width}, {height})")
        
        def on_key(event):
            """Finish selection on Enter key"""
            if event.key == 'enter':
                plt.close()
                print(f"\nSelected {len(rois)} ROIs")
        
        # Create selector
        rs = EllipseSelector(ax, onselect, useblit=True,
                             button=[1], minspanx=5, minspany=5,
                             spancoords='pixels', interactive=True)
        
        fig.canvas.mpl_connect('key_press_event', on_key)
        plt.show()
        
        if rois:
            self.extract_manual_traces(movie, rois)
            print(f"Successfully extracted traces from {len(rois)} manual ROIs")
        else:
            print("No ROIs selected manually.")
    
    def extract_manual_traces(self, movie, rois):
        """
        Extract fluorescence traces from manual ROIs.
        
        Parameters
        ----------
        movie : np.ndarray
            Loaded movie data
        rois : list
            List of ROI coordinates (y, x, height, width)
        """
        height, width = movie.shape[1], movie.shape[2]
        masks = []
        
        # Create binary masks from ROI coordinates
        for i, (y, x, h, w) in enumerate(rois):
            mask = np.zeros((height, width), dtype=bool)
            # Ensure coordinates are within bounds
            y_end = min(y + h, height)
            x_end = min(x + w, width)
            mask[y:y_end, x:x_end] = True
            masks.append(mask)
        
        masks_array = np.array(masks)
        
        # Extract traces
        F_manual = []
        for mask in masks_array:
            trace = np.mean(movie[:, mask], axis=1)
            F_manual.append(trace)
        
        self.manual_traces = np.array(F_manual)
        self.manual_masks = masks_array
        
        # Create dummy Suite2p-compatible outputs for compatibility
        n_rois = len(rois)
        n_frames = movie.shape[0]
        
        self.F = self.manual_traces
        self.Fneu = np.zeros_like(self.manual_traces)  # No neuropil for manual
        self.spks = np.zeros_like(self.manual_traces)  # No spikes for manual
        self.iscell = np.column_stack([np.ones(n_rois), np.ones(n_rois)])  # All are cells
        self.stat = self.create_dummy_stat(masks_array)
        
        print(f"Manual traces shape: {self.manual_traces.shape}")
    
    def create_dummy_stat(self, masks):
        """
        Create dummy stat structure for manual ROIs to maintain compatibility.
        """
        stat = []
        for mask in masks:
            ypix, xpix = np.where(mask)
            stat.append({
                'ypix': ypix,
                'xpix': xpix,
                'lam': np.ones(len(ypix)) / len(ypix),  # Uniform weights
                'med': [np.median(ypix), np.median(xpix)],  # Center
                'footprint': 1  # Simple footprint
            })
        return np.array(stat, dtype=object)
    
    def load_results(self):
        """Load Suite2p output files."""
        result_path = self.output_dir / 'suite2p' / 'plane0'
        
        if not result_path.exists():
            raise FileNotFoundError(f"Suite2p results not found at {result_path}")
        
        print(f"\nLoading results from {result_path}")
        
        # Load main outputs
        self.stat = np.load(result_path / 'stat.npy', allow_pickle=True)
        self.F = np.load(result_path / 'F.npy')
        self.Fneu = np.load(result_path / 'Fneu.npy')
        self.spks = np.load(result_path / 'spks.npy')
        self.iscell = np.load(result_path / 'iscell.npy')
        self.ops = np.load(result_path / 'ops.npy', allow_pickle=True).item()
        
        n_cells = np.sum(self.iscell[:, 0] > 0)
        n_total = len(self.iscell)
        
        print(f"Loaded {n_total} ROIs ({n_cells} classified as cells)")
        print(f"Traces shape: {self.F.shape} (ROIs × frames)")
    
    def get_corrected_traces(self, neuropil_coefficient=0.7, cells_only=True):
        """
        Get neuropil-corrected fluorescence traces.
        Works for both automatic and manual modes.
        """
        if self.manual_mode:
            # For manual mode, just return the raw traces
            print("Using manual ROI traces (no neuropil correction available)")
            return self.manual_traces
        else:
            # For automatic mode, use neuropil correction
            if self.F is None:
                raise ValueError("Run Suite2p first or load existing results!")
            
            F_corrected = self.F - neuropil_coefficient * self.Fneu
            
            if cells_only:
                cell_mask = self.iscell[:, 0] > 0
                F_corrected = F_corrected[cell_mask]
                print(f"Returning {np.sum(cell_mask)} cells (filtered non-cells)")
            
            return F_corrected
    
    def visualize_rois(self, max_display=50, save_path=None):
        """
        Visualize detected ROIs.
        Works for both automatic and manual modes.
        """
        if self.manual_mode:
            # Manual mode visualization
            movie_path = self.data_path / self.tiff_file
            movie = tifffile.imread(movie_path)
            mean_img = np.mean(movie, axis=0)
            
            fig, axes = plt.subplots(1, 2, figsize=(12, 5))
            
            # Mean image
            axes[0].imshow(mean_img, cmap='gray')
            axes[0].set_title('Mean Image')
            axes[0].axis('off')
            
            # ROI overlay
            axes[1].imshow(mean_img, cmap='gray')
            for i, mask in enumerate(self.manual_masks):
                if i >= max_display:
                    break
                # Plot outline
                ypix, xpix = np.where(mask)
                if len(ypix) > 0:
                    y_min, y_max = ypix.min(), ypix.max()
                    x_min, x_max = xpix.min(), xpix.max()
                    axes[1].plot([x_min, x_max, x_max, x_min, x_min],
                               [y_min, y_min, y_max, y_max, y_min], 
                               'r-', linewidth=1, alpha=0.8, label=f'ROI {i+1}')
            
            axes[1].set_title(f'Manual ROIs ({len(self.manual_masks)} cells)')
            axes[1].axis('off')
            axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            
        else:
            # Automatic mode visualization (your original code)
            if self.stat is None:
                raise ValueError("No ROIs loaded!")
            
            result_path = self.output_dir / 'suite2p' / 'plane0'
            mean_img = np.load(result_path / 'ops.npy', allow_pickle=True).item()['meanImg']
            
            roi_img = np.zeros_like(mean_img)
            cell_mask = self.iscell[:, 0] > 0
            n_cells = np.sum(cell_mask)
            
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            axes[0].imshow(mean_img, cmap='gray')
            axes[0].set_title('Mean Image')
            axes[0].axis('off')
            
            for i, (st, is_cell) in enumerate(zip(self.stat, self.iscell)):
                if i >= max_display:
                    break
                if is_cell[0] > 0:
                    ypix = st['ypix']
                    xpix = st['xpix']
                    roi_img[ypix, xpix] = i + 1
            
            axes[1].imshow(roi_img, cmap='nipy_spectral', alpha=0.8)
            axes[1].set_title(f'ROIs ({n_cells} cells)')
            axes[1].axis('off')
            
            axes[2].imshow(mean_img, cmap='gray')
            for i, (st, is_cell) in enumerate(zip(self.stat, self.iscell)):
                if i >= max_display:
                    break
                if is_cell[0] > 0:
                    ypix = st['ypix']
                    xpix = st['xpix']
                    y_min, y_max = ypix.min(), ypix.max()
                    x_min, x_max = xpix.min(), xpix.max()
                    axes[2].plot([x_min, x_max, x_max, x_min, x_min],
                               [y_min, y_min, y_max, y_max, y_min], 
                               'r-', linewidth=0.5, alpha=0.7)
            
            axes[2].set_title('Overlay')
            axes[2].axis('off')
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            mode = "manual" if self.manual_mode else "automatic"
            print(f"Saved {mode} ROI visualization to {save_path}")
        
        plt.show()
        return fig
        
    def export_results(self, export_dir='./exported_results'):
        """
        Export results in easy-to-use format.
        Works for both automatic and manual modes.
        """
        export_dir = Path(export_dir)
        export_dir.mkdir(exist_ok=True)
        
        if self.manual_mode:
            # Manual mode export
            F_corrected = self.manual_traces
            dff = self.compute_dff(F_corrected)
            
            # Save as numpy arrays
            np.save(export_dir / 'fluorescence_manual.npy', F_corrected)
            np.save(export_dir / 'dff_manual.npy', dff)
            
            # Save ROI masks
            np.save(export_dir / 'roi_masks.npy', self.manual_masks)
            
            # Save metadata
            metadata = {
                'frame_rate': getattr(self.ops, 'fs', 300) if self.ops else 300,
                'n_cells': F_corrected.shape[0],
                'n_frames': F_corrected.shape[1],
                'mode': 'manual',
                'export_time': str(np.datetime64('now'))
            }
            np.save(export_dir / 'metadata.npy', metadata)
            
            print(f"\nExported MANUAL results to {export_dir}:")
            print(f"  - fluorescence_manual.npy: {F_corrected.shape}")
            print(f"  - roi_masks.npy: {len(self.manual_masks)} masks")
            
        else:
            # Automatic mode export (your original code)
            F_corrected = self.get_corrected_traces(cells_only=True)
            dff = self.compute_dff(F_corrected)
            spks = self.spks[self.iscell[:, 0] > 0]
            
            np.save(export_dir / 'fluorescence_corrected.npy', F_corrected)
            np.save(export_dir / 'dff.npy', dff)
            np.save(export_dir / 'spikes_deconvolved.npy', spks)
            
            cells_stat = self.stat[self.iscell[:, 0] > 0]
            np.save(export_dir / 'cell_stats.npy', cells_stat)
            
            metadata = {
                'frame_rate': self.ops['fs'],
                'n_cells': F_corrected.shape[0],
                'n_frames': F_corrected.shape[1],
                'diameter': self.ops['diameter'],
                'tau': self.ops['tau'],
                'mode': 'automatic'
            }
            np.save(export_dir / 'metadata.npy', metadata)
            
            print(f"\nExported AUTOMATIC results to {export_dir}:")
            print(f"  - fluorescence_corrected.npy: {F_corrected.shape}")
            print(f"  - dff.npy: {dff.shape}")
            print(f"  - spikes_deconvolved.npy: {spks.shape}")
            print(f"  - cell_stats.npy: {len(cells_stat)} cells")
        
        print(f"  - metadata.npy")
        
        return export_dir


# =============================================================================
# UPDATED COMPLETE USAGE EXAMPLE
# =============================================================================

def run_complete_voltage_pipeline(tiff_path, frame_rate=1000, diameter=12, enable_manual_fallback=True):
    """
    One-function pipeline for voltage imaging with automatic manual fallback.
    
    Parameters
    ----------
    tiff_path : str
        Path to your denoised TIFF file
    frame_rate : float
        Imaging frame rate in Hz
    diameter : int
        Expected neuron diameter in pixels
    enable_manual_fallback : bool
        If True, automatically use manual selection when no cells are found
        
    Returns
    -------
    pipeline : Suite2pVoltagePipeline
        Pipeline object with all results
    """
    # Parse path
    tiff_path = Path(tiff_path)
    data_path = tiff_path.parent
    tiff_file = tiff_path.name
    
    print("="*60)
    print("SUITE2P VOLTAGE IMAGING PIPELINE WITH MANUAL FALLBACK")
    print("="*60)
    
    # Initialize
    pipeline = Suite2pVoltagePipeline(
        data_path=data_path,
        tiff_file=tiff_file,
        output_dir=data_path / 'suite2p_output'
    )
    
    # Configure for voltage imaging
    pipeline.configure_ops(
        frame_rate=frame_rate,
        diameter=diameter,
        tau=0.0028,
        spatial_hp=100,
        threshold_scaling=1.0
    )
    
    # Run Suite2p with automatic manual fallback
    pipeline.run_suite2p_with_manual_fallback(enable_manual_fallback=enable_manual_fallback)
    
    # Visualize results
    mode = "MANUAL" if pipeline.manual_mode else "AUTOMATIC"
    pipeline.visualize_rois(save_path=data_path / f'suite2p_rois_{mode.lower()}.png')
    pipeline.plot_traces(n_neurons=10, trace_type='dff', 
                        save_path=data_path / f'suite2p_traces_{mode.lower()}.png')
    
    # Export results
    pipeline.export_results(export_dir=data_path / 'suite2p_exported')
    
    print("\n" + "="*60)
    print(f"PIPELINE COMPLETE! (Mode: {mode})")
    print("="*60)
    
    return pipeline


if __name__ == "__main__":
    # SIMPLE ONE-LINE USAGE WITH MANUAL FALLBACK:
    pipeline = run_complete_voltage_pipeline(
        tiff_path='D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/014-SD10hz-70_denoised_full.tif',
        frame_rate=1000,
        diameter=75,
        enable_manual_fallback=True  # This enables the automatic fallback to manual selection
    )
    
    # Access results (works for both automatic and manual modes):
    F_corrected = pipeline.get_corrected_traces()
    dff = pipeline.compute_dff()
    if not pipeline.manual_mode:  # Only available in automatic mode
        spikes = pipeline.spks

SUITE2P VOLTAGE IMAGING PIPELINE WITH MANUAL FALLBACK
Suite2p configured for voltage imaging:
  Frame rate: 1000 Hz
  Tau: 0.0028 s
  Diameter: 75 pixels
  Spatial high-pass: 100 pixels

Running Suite2p...
Input: D:\Paris2025\ForceB Undiluted\20251001\cell2\014-SD10hz-70\014-SD10hz-70_denoised_full.tif
Output: D:\Paris2025\ForceB Undiluted\20251001\cell2\014-SD10hz-70\suite2p_output
{'data_path': ['D:\\Paris2025\\ForceB Undiluted\\20251001\\cell2\\014-SD10hz-70'], 'tiff_list': ['D:\\Paris2025\\ForceB Undiluted\\20251001\\cell2\\014-SD10hz-70\\014-SD10hz-70_denoised_full.tif'], 'save_path0': 'D:\\Paris2025\\ForceB Undiluted\\20251001\\cell2\\014-SD10hz-70\\suite2p_output'}
FOUND BINARIES AND OPS IN ['D:\\Paris2025\\ForceB Undiluted\\20251001\\cell2\\014-SD10hz-70\\suite2p_output\\suite2p\\plane0\\ops.npy']
removing previous detection and extraction files, if present
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, ops['do_registration']=0
binary path:

Traceback (most recent call last):
  File "C:\Users\sofik\AppData\Local\Temp\ipykernel_23324\569607799.py", line 108, in run_suite2p_with_manual_fallback
    output_ops = run_s2p(ops=self.ops, db=db)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sofik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\suite2p\run_s2p.py", line 550, in run_s2p
    op = run_plane(op, ops_path=ops_path)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sofik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\suite2p\run_s2p.py", line 342, in run_plane
    ops = pipeline(f_reg, f_raw, f_reg_chan2, f_raw_chan2, run_registration, ops,
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sofik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Pyt

Manual traces shape: (1, 1481)
Successfully extracted traces from 1 manual ROIs

Processing complete!
Saved manual ROI visualization to D:\Paris2025\ForceB Undiluted\20251001\cell2\014-SD10hz-70\suite2p_rois_manual.png
Using manual ROI traces (no neuropil correction available)
Saved traces to D:\Paris2025\ForceB Undiluted\20251001\cell2\014-SD10hz-70\suite2p_traces_manual.png

Exported MANUAL results to D:\Paris2025\ForceB Undiluted\20251001\cell2\014-SD10hz-70\suite2p_exported:
  - fluorescence_manual.npy: (1, 1481)
  - dff_manual.npy: (1, 1481)
  - roi_masks.npy: 1 masks
  - metadata.npy

PIPELINE COMPLETE! (Mode: MANUAL)
Using manual ROI traces (no neuropil correction available)
Using manual ROI traces (no neuropil correction available)


## Load ABF file 

In [ ]:
import datetime
from matplotlib.ticker import MaxNLocator
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import pyabf
from scipy.signal import butter, filtfilt, welch, iirnotch

print("Current Directory:", os.getcwd())
read_abf_path = r"C:\Users\sofik\.vscode\Voltage_imaging\data_io"
print("functions_path:", read_abf_path)
sys.path.append(read_abf_path)

# Check if read_abf.py exists at this location
if os.path.isfile(os.path.join(read_abf_path, "read_abf.py")):
    print("read_abf.py found at:", read_abf_path)
else:
    print("read_abf.py NOT found at:", read_abf_path)

# Now import Abfdata from functions
import read_abf as functions 


def process_abf_data(file_path, color, label):
    # Load the data
    data = functions.Abfdata(file_path)
    base_name = os.path.splitext(os.path.basename(file_path))[0]

    # Extract trace and time data
    trace_data = data.extract_trace_data()
    time_values = data.get_time_values() * 1000  # convert to ms

    #EXCLUDE INDICES IF NEEDED 
    excluded_indices = {}
    filtered_indices = [idx for idx in range(len(trace_data)) if idx not in excluded_indices]

    # Create a Gaussian window for filtering
    std_dev = 5
    window_size = 10
    window = signal.windows.gaussian(window_size, std_dev)
    filtered_trace_data = trace_data[filtered_indices]

    plt.figure(figsize=(4, 4))

    # Plot pulse data
    pulse_data = [data.extract_pulse_data() for _ in filtered_trace_data]
    print("Pulse data length:", len(pulse_data))
    print("Pulse data shape:", np.array(pulse_data).shape)
    print("Pulse data type:", type(pulse_data))
    print("Pulse data:", pulse_data)
    pulse_data = np.array(pulse_data)
    stim_time = []
    for pulse in pulse_data:
        #plt.plot(time_values, np.array(pulse), color='magenta', alpha=0.9)
        # Find peaks in the data
        peaks, props = signal.find_peaks(pulse, height=2.1, width=1)  # adjust height as needed
        pulse_starts = time_values[props["left_ips"].astype(int)]
        pulse_ends   = time_values[props["right_ips"].astype(int)]
        #for start, end in zip(pulse_starts, pulse_ends):
        #    plt.vlines([start, end], ymin=-200, ymax=200, color="paleturquoise", alpha=0.3)
        stim_times = time_values[peaks]
    stim_time.append(stim_times)


    filt_trace_data = []
    for sweep_data in filtered_trace_data:
        filtered_sweep_data = signal.convolve(sweep_data, window, mode='same') / sum(window)
        filt_trace_data.append(filtered_sweep_data)

    # Average sweeps and apply baseline correction
    averaged_data = data.average_abf_sweeps()
    window = signal.windows.gaussian(5, 2)
    averaged_data_data = signal.convolve(averaged_data, window, mode='same') / sum(window)
    baseline_averageddata = data.baseline_correction(averaged_data_data)
    
    fs = 50000  # Replace with your actual sampling rate
    f, Pxx = welch(baseline_averageddata, fs, nperseg=2048)
    f0 = 50  # Notch filter frequency (Hz)
    Q = 300   # Quality factor
    b, a = iirnotch(f0, Q, fs)
    cleaned_trace = filtfilt(b, a, baseline_averageddata)

    def butter_lowpass(cutoff, fs, order=4):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        return butter(order, normal_cutoff, btype='low', analog=False)

    cutoff = 20  # or lower, depending on what you want to keep
    b, a = butter_lowpass(cutoff, fs)
    smoothed_trace = filtfilt(b, a, cleaned_trace)

    #COMMENT THIS OUT IF YOU DON'T WANT TO SEE ALL THE TRACES 
    baseline_cor_data = data.baseline_correction(filt_trace_data)
    for i, sweep_data in enumerate(baseline_cor_data[:]):
       plt.plot(time_values, sweep_data, alpha=.8, color=color)

    # Plot averaged trace
    #peak = []
    #plt.plot(time_values, smoothed_trace, label=f'{label}', color='darkcyan', alpha=0.8, linewidth=1), 
    #peaks, _ = signal.find_peaks(baseline_averageddata, height=90,prominence=8)
    #peak.append(peaks)

    #contour_heights = baseline_averageddata[peaks] - prominences
    #plt.plot(peaks, baseline_averageddata[peaks], "x")
    plt.xlim(100,1000)
    plt.ylim(-5, 80)
    #print("Averaged data peaks:", peaks)
    plt.show()
    # Compute more peak details from averaged data
    peak_indices, peak_props = signal.find_peaks(baseline_averageddata, height=90, prominence=8)

    peak_times = time_values[peak_indices]
    peak_amps = baseline_averageddata[peak_indices]
    baseline_value = 0  # Or compute your baseline from data if needed

    # Duration as width at half prominence
    widths, width_heights, left_ips, right_ips = signal.peak_widths(baseline_averageddata, peak_indices, rel_height=0.5)
    durations = (right_ips - left_ips) * (time_values[1] - time_values[0])  # in ms

    # Replace your return with this:
    return {
        "averaged_data": averaged_data_data,
        "time_values": time_values,
        "baseline_averaged": baseline_averageddata,
        "stim_times": stim_times,
        "stim_t": stim_time,
    }

In [221]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Load data ---
export_dir = Path('D:/Paris2025/ForceB Undiluted/20251001/cell2/014-SD10hz-70/suite2p_exported')
F = np.load(export_dir / 'fluorescence_manual.npy')  # shape (1, 1485)
F_trace = F.squeeze()         # now shape (1485,)

# --- 2. Define frame ranges ---
# adjust these according to your experiment timing
light_off_frames = np.arange(1000, 1200)      # frames with lights off
light_on_frames = np.arange(100, 200)     # frames with lights on

# --- 3. Compute baselines safely ---
F_off = np.mean(F_trace[light_off_frames])
F_on_baseline = np.mean(F_trace[light_on_frames])  # first ~50 frames after light on

# --- 4. Compute ΔF/F₀ ---
F0 = F_on_baseline - F_off 
dF = F_trace - F_on_baseline
dF_over_F0 = (dF / F0) *100  # convert to percentage

# --- 5. Optional: Replace NaNs and plot ---
dF_over_F0 = np.nan_to_num(dF_over_F0, nan=0.0)

plt.figure(figsize=(10, 4))
plt.plot(dF_over_F0, label='ΔF/F₀')
#plt.axvspan(light_off_frames[0], light_off_frames[-1], color='gray', alpha=0.2, label='Light off')
#plt.axvspan(light_on_frames[0], light_on_frames[-1], color='yellow', alpha=0.1, label='Light on')
plt.xlabel('Frame')
plt.ylabel('ΔF/F₀')
plt.xlim(50, 900)
plt.ylim(-10, 20)
process_abf_data("D:/Paris2025/Ephys/011025/2025_10_01_0014.abf", "darkcyan", "ForceB")
plt.legend()
plt.tight_layout()
plt.show()


[-62.6221 -62.6221 -62.6221 ... -62.0728 -62.1033 -62.1033]
Pulse data length: 1
Pulse data shape: (1, 75000)
Pulse data type: <class 'list'>
Pulse data: [array([-62.6221, -62.6221, -62.6221, ..., -62.0728, -62.1033, -62.1033],
      shape=(75000,), dtype=float32)]


In [ ]:
# Load data
fluorescence = np.load(export_dir / 'fluorescence_manual.npy')
dff = np.load(export_dir / 'dff_manual.npy')

# Compute SNR in one line
noise = np.std(F_trace[light_on_frames])
signal = np.max(F_trace[30:500]) - F_on_baseline
snr = (signal / noise) 

print(f"SNR: {snr}")
print(f"Mean SNR: {np.mean(snr):.2f}")
F_trace[50:500]

SNR: 14.673527717590332
Mean SNR: 14.67
